# 출발지 GPS

In [ ]:
import serial
import requests
import re
import time

# ✅ 위도/경도 변환 함수는 그대로 유지
def convert_to_decimal(degree_str, direction):
    if not degree_str:
        return None
    if direction in ['N', 'S']:  # 위도
        d, m = int(degree_str[:2]), float(degree_str[2:])
    elif direction in ['E', 'W']:  # 경도는 3자리
        d, m = int(degree_str[:3]), float(degree_str[3:])
    else:
        return None
    result = d + m / 60
    if direction in ['S', 'W']:
        result *= -1
    return result

# ✅ 시리얼 GPS 파싱 함수는 유지
def get_gps_coords(ser, timeout_sec=10):
    start_time = time.time()
    while time.time() - start_time < timeout_sec:
        line = ser.readline().decode('utf-8', errors='ignore').strip()
        if line.startswith('$GNRMC') or line.startswith('$GNGGA'):
            try:
                parts = line.split(',')
                if parts[0] == '$GNRMC' and parts[2] == 'A':
                    lat = convert_to_decimal(parts[3], parts[4])
                    lon = convert_to_decimal(parts[5], parts[6])
                    return lat, lon
                elif parts[0] == '$GNGGA' and parts[6] in ['1', '2']:
                    lat = convert_to_decimal(parts[2], parts[3])
                    lon = convert_to_decimal(parts[4], parts[5])
                    return lat, lon
            except Exception:
                continue
    print("❌ GPS 유효 데이터 수신 실패 (타임아웃)")
    return None, None

# ✅ 주소 → 위경도 변환
def address_to_coord_google(address, google_api_key):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": address, "key": google_api_key}
    res = requests.get(url, params=params).json()
    try:
        loc = res["results"][0]["geometry"]["location"]
        return loc["lat"], loc["lng"]
    except:
        print(f"❌ 주소 변환 실패: {address}")
        return None, None

# ✅ Tmap 보행자 경로 요청
def get_tmap_pedestrian_steps(start_lat, start_lon, end_lat, end_lon, app_key):
    url = "https://apis.openapi.sk.com/tmap/routes/pedestrian"
    headers = {"appKey": app_key, "Content-Type": "application/json"}
    body = {
        "startX": str(start_lon), "startY": str(start_lat),
        "endX": str(end_lon), "endY": str(end_lat),
        "reqCoordType": "WGS84GEO", "resCoordType": "WGS84GEO",
        "startName": "출발지", "endName": "도착지"
    }
    res = requests.post(url, headers=headers, json=body).json()
    steps = []
    if "features" not in res:
        print("❌ Tmap 경로 요청 실패:", res)
        return steps
    for feat in res["features"]:
        desc = feat["properties"].get("description")
        if desc:
            steps.append(desc)
    return steps

# ✅ Google + Tmap 대중교통 안내
def get_combined_route_script_detailed(start_lat, start_lon, end_lat, end_lon, google_api_key, tmap_key, start_name, end_name):
    g_url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": f"{start_lat},{start_lon}",
        "destination": f"{end_lat},{end_lon}",
        "mode": "transit", "language": "ko", "key": google_api_key
    }
    res = requests.get(g_url, params=params).json()
    if res["status"] != "OK":
        print("❌ Google 경로 요청 실패:", res["status"])
        return []

    leg = res["routes"][0]["legs"][0]
    duration = leg["duration"]["text"]
    total_script = []
    bus_lines = []

    for step in leg["steps"]:
        if step["travel_mode"] == "TRANSIT":
            line = step["transit_details"]["line"].get("short_name", step["transit_details"]["line"].get("name"))
            if line not in bus_lines:
                bus_lines.append(line)

    summary = f"{start_name}부터 {end_name}까지 총 소요시간은 {duration}입니다. "
    if bus_lines:
        summary += "탑승할 노선은 " + ", ".join([f"{b}번" for b in bus_lines]) + "입니다. "
    summary += "안내를 시작합니다."
    total_script.append(summary)

    step_num = 1
    for step in leg["steps"]:
        mode = step["travel_mode"]
        dist = step["distance"]["text"]
        dur = step["duration"]["text"]
        if mode == "WALKING":
            s = step["start_location"]
            e = step["end_location"]
            tmap_steps = get_tmap_pedestrian_steps(s["lat"], s["lng"], e["lat"], e["lng"], tmap_key)
            for desc in tmap_steps:
                desc_clean = desc.strip()
                if not re.fullmatch(r'[.,\s\d\w가-힣]*[,.]\s*\d+m', desc_clean) and desc_clean:
                    total_script.append(f"{step_num}. {desc_clean} ({dist}, {dur})")
                    step_num += 1
        elif mode == "TRANSIT":
            t = step["transit_details"]
            line = t["line"].get("short_name", t["line"].get("name", "노선 없음"))
            vehicle = t["line"]["vehicle"]["name"]
            dep = t["departure_stop"]["name"]
            arr = t["arrival_stop"]["name"]
            dep_time = t["departure_time"]["text"]
            arr_time = t["arrival_time"]["text"]
            stops = t["num_stops"]
            total_script.append(
                f"{step_num}. {dep} 정류장에서 {line}번 {vehicle}을({dep_time} 출발) 탑승하여 {stops}개 정거장을 이동하고, {arr_time}에 {arr} 정류장에서 하차합니다. 예상 소요 시간은 {dur}입니다."
            )
            step_num += 1

    return total_script

# ✅ 최종 실행 함수

def run_combined_route_system_tts(google_api_key, tmap_api_key):
    end_name = input("도착지를 입력하세요 (예: 광운대): ").strip()
    mode_choice = input("어떤 방식으로 이동할까요? (1. 대중교통 or 2. 도보): ").strip()

    print("\n📡 현재 GPS 위치를 가져오는 중...")
    try:
        ser = serial.Serial("COM3", 9600, timeout=1)  # 포트 번호 필요시 변경
    except serial.SerialException as e:
        print(f"❌ 시리얼 포트 열기 실패: {e}")
        return

    start_lat, start_lon = get_gps_coords(ser)
    ser.close()

    if not start_lat:
        print("❌ GPS 위치 획득 실패")
        return

    print(f"📍 현재 위치: 위도 {start_lat}, 경도 {start_lon}")

    end_lat, end_lon = address_to_coord_google(end_name, google_api_key)
    if not end_lat:
        print("❌ 도착지 좌표 변환 실패")
        return

    if mode_choice == "1":
        print(f"\n📍 현재 위치부터 {end_name}까지 대중교통 경로 안내를 시작합니다.")
        script = get_combined_route_script_detailed(
            start_lat, start_lon, end_lat, end_lon,
            google_api_key, tmap_api_key,
            "현재 위치", end_name
        )
        for line in script:
            print(line)
    elif mode_choice == "2":
        print(f"\n📍 현재 위치부터 {end_name}까지 도보 경로 안내를 시작합니다.")
        steps = get_tmap_pedestrian_steps(start_lat, start_lon, end_lat, end_lon, tmap_api_key)
        for i, desc in enumerate(steps, 1):
            print(f"{i}. {desc}")
    else:
        print("❌ 잘못된 선택입니다. 1 또는 2를 입력하세요.")


In [ ]:
google_api_key = "abcdef1234567890abcdef1234"  # Google Maps API 키
tmap_api_key = "abcdef1234567890abcdef1234"

run_combined_route_system_tts(google_api_key, tmap_api_key)



📡 현재 GPS 위치를 가져오는 중...
📍 현재 위치: 위도 37.620438, 경도 127.06076133333333

📍 현재 위치부터 청량리역까지 도보 경로 안내를 시작합니다.
1. 보행자도로 을 따라 16m 이동
2. 보행자도로, 16m
3. 월계어린이공원 에서 좌회전 후 159m 이동 
4. , 159m
5. 동신카세차장 에서 우회전 후 석계로 을 따라 602m 이동 
6. 석계로, 602m
7. 성신한의원 에서 좌측 횡단보도 후 보행자도로 을 따라 11m 이동 
8. 보행자도로, 11m
9. 성신한의원 에서 우회전 후 석계로 을 따라 54m 이동 
10. 석계로, 54m
11. SHOW 에서 우측 횡단보도 후 보행자도로 을 따라 16m 이동 
12. 보행자도로, 16m
13. STCO 석계점 에서 직진 후 화랑로 을 따라 247m 이동 
14. 화랑로, 247m
15. 이훈헤어칼라 석계점 에서 좌측 횡단보도 후 보행자도로 을 따라 33m 이동 
16. 보행자도로, 33m
17. DIGITALLG 에서 우회전 후 화랑로 을 따라 166m 이동 
18. 화랑로, 166m
19. 오일뱅크 우리 에서 횡단보도 후 보행자도로 을 따라 36m 이동 
20. 보행자도로, 36m
21. 대게축제 에서 직진 후 화랑로 을 따라 360m 이동 
22. 화랑로, 360m
23. 돌곶이로, 12m
24. 돌곶이역  6번출구 에서 횡단보도 후 보행자도로 을 따라 20m 이동 
25. 보행자도로, 20m
26. 돌곶이역  7번출구 에서 좌회전 후 돌곶이로 을 따라 521m 이동 
27. 돌곶이로, 521m
28. 이문로, 486m
29. , 9m
30. T world 유니쿱점 에서 횡단보도 후 보행자도로 을 따라 21m 이동 
31. 보행자도로, 21m
32. 유진문구 에서 직진 후 1393m 이동 
33. , 1393m
34. 횡단보도 후 5m 이동 
35. , 5m
36. 직진 후 이문로 을 따라 110m 이동 
37. 이문로, 110m
38. 회기로, 5m
39. 

# 지도 표시

In [17]:
import serial
import requests
import re
import time
import urllib.parse

def convert_to_decimal(degree_str, direction):
    if not degree_str:
        return None
    if direction in ['N', 'S']:
        d, m = int(degree_str[:2]), float(degree_str[2:])
    elif direction in ['E', 'W']:
        d, m = int(degree_str[:3]), float(degree_str[3:])
    else:
        return None
    result = d + m / 60
    if direction in ['S', 'W']:
        result *= -1
    return result

def get_gps_coords(ser, timeout_sec=10):
    start_time = time.time()
    while time.time() - start_time < timeout_sec:
        line = ser.readline().decode('utf-8', errors='ignore').strip()
        if line.startswith('$GNRMC') or line.startswith('$GNGGA'):
            try:
                parts = line.split(',')
                if parts[0] == '$GNRMC' and parts[2] == 'A':
                    lat = convert_to_decimal(parts[3], parts[4])
                    lon = convert_to_decimal(parts[5], parts[6])
                    return lat, lon
                elif parts[0] == '$GNGGA' and parts[6] in ['1', '2']:
                    lat = convert_to_decimal(parts[2], parts[3])
                    lon = convert_to_decimal(parts[4], parts[5])
                    return lat, lon
            except Exception:
                continue
    print("❌ GPS 유효 데이터 수신 실패 (타임아웃)")
    return None, None

def address_to_coord_google(address, google_api_key):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": address, "key": google_api_key}
    res = requests.get(url, params=params).json()
    try:
        loc = res["results"][0]["geometry"]["location"]
        return loc["lat"], loc["lng"]
    except:
        print(f"❌ 주소 변환 실패: {address}")
        return None, None

def get_tmap_pedestrian_steps(start_lat, start_lon, end_lat, end_lon, app_key):
    url = "https://apis.openapi.sk.com/tmap/routes/pedestrian"
    headers = {"appKey": app_key, "Content-Type": "application/json"}
    body = {
        "startX": str(start_lon), "startY": str(start_lat),
        "endX": str(end_lon), "endY": str(end_lat),
        "reqCoordType": "WGS84GEO", "resCoordType": "WGS84GEO",
        "startName": "출발지", "endName": "도착지"
    }
    res = requests.post(url, headers=headers, json=body).json()
    steps = []
    if "features" not in res:
        print("❌ Tmap 경로 요청 실패:", res)
        return steps
    for feat in res["features"]:
        desc = feat["properties"].get("description")
        if desc:
            steps.append(desc)
    return steps

def get_combined_route_script_detailed(start_lat, start_lon, end_lat, end_lon, google_api_key, tmap_key, start_name, end_name):
    g_url = "https://maps.googleapis.com/maps/api/directions/json"
    params = {
        "origin": f"{start_lat},{start_lon}",
        "destination": f"{end_lat},{end_lon}",
        "mode": "transit", "language": "ko", "key": google_api_key
    }
    res = requests.get(g_url, params=params).json()
    if res["status"] != "OK":
        print("❌ Google 경로 요청 실패:", res["status"])
        return []

    leg = res["routes"][0]["legs"][0]
    duration = leg["duration"]["text"]
    total_script = []
    bus_lines = []

    for step in leg["steps"]:
        if step["travel_mode"] == "TRANSIT":
            line = step["transit_details"]["line"].get("short_name", step["transit_details"]["line"].get("name"))
            if line not in bus_lines:
                bus_lines.append(line)

    summary = f"{start_name}부터 {end_name}까지 총 소요시간은 {duration}입니다. "
    if bus_lines:
        summary += "탑승할 노선은 " + ", ".join([f"{b}번" for b in bus_lines]) + "입니다. "
    summary += "안내를 시작합니다."
    total_script.append(summary)

    step_num = 1
    for step in leg["steps"]:
        mode = step["travel_mode"]
        dist = step["distance"]["text"]
        dur = step["duration"]["text"]
        if mode == "WALKING":
            s = step["start_location"]
            e = step["end_location"]
            tmap_steps = get_tmap_pedestrian_steps(s["lat"], s["lng"], e["lat"], e["lng"], tmap_key)
            for desc in tmap_steps:
                desc_clean = desc.strip()
                if not re.fullmatch(r'[.,\s\d\w가-힣]*[,.]\s*\d+m', desc_clean) and desc_clean:
                    total_script.append(f"{step_num}. {desc_clean} ({dist}, {dur})")
                    step_num += 1
        elif mode == "TRANSIT":
            t = step["transit_details"]
            line = t["line"].get("short_name", t["line"].get("name", "노선 없음"))
            vehicle = t["line"]["vehicle"]["name"]
            dep = t["departure_stop"]["name"]
            arr = t["arrival_stop"]["name"]
            dep_time = t["departure_time"]["text"]
            arr_time = t["arrival_time"]["text"]
            stops = t["num_stops"]
            total_script.append(
                f"{step_num}. {dep} 정류장에서 {line}번 {vehicle}을({dep_time} 출발) 탑승하여 {stops}개 정거장을 이동하고, {arr_time}에 {arr} 정류장에서 하차합니다. 예상 소요 시간은 {dur}입니다."
            )
            step_num += 1

    return total_script

def run_combined_route_system_tts(google_api_key, tmap_api_key):
    end_name = "광운대"  # 자동 입력
    mode_choice = "1"  # 자동으로 대중교통 모드 실행

    print("\n📡 현재 GPS 위치를 가져오는 중...")
    try:
        ser = serial.Serial("COM3", 9600, timeout=1)
    except serial.SerialException as e:
        print(f"❌ 시리얼 포트 열기 실패: {e}")
        return

    start_lat, start_lon = get_gps_coords(ser)
    ser.close()

    if not start_lat:
        print("❌ GPS 위치 획득 실패")
        return

    print(f"📍 현재 위치: 위도 {start_lat}, 경도 {start_lon}")

    end_lat, end_lon = address_to_coord_google(end_name, google_api_key)
    if not end_lat:
        print("❌ 도착지 좌표 변환 실패")
        return

    if mode_choice == "1":
        print(f"\n📍 현재 위치부터 {end_name}까지 대중교통 경로 안내를 시작합니다.")
        script = get_combined_route_script_detailed(
            start_lat, start_lon, end_lat, end_lon,
            google_api_key, tmap_api_key,
            "현재 위치", end_name
        )
        for line in script:
            print(line)

        # ✅ 지도 링크 출력
        origin = f"{start_lat},{start_lon}"
        dest = f"{end_lat},{end_lon}"
        link = f"https://www.google.com/maps/dir/{urllib.parse.quote(origin)}/{urllib.parse.quote(dest)}"
        print(f"\n🗺️ 경로를 지도에서 확인하려면 아래 링크를 여세요:\n{link}")

    elif mode_choice == "2":
        print(f"\n📍 현재 위치부터 {end_name}까지 도보 경로 안내를 시작합니다.")
        steps = get_tmap_pedestrian_steps(start_lat, start_lon, end_lat, end_lon, tmap_api_key)
        for i, desc in enumerate(steps, 1):
            print(f"{i}. {desc}")


In [ ]:
google_api_key = "abcdef1234567890abcdef1234"  # Google Maps API 키
tmap_api_key = "abcdef1234567890abcdef1234"

run_combined_route_system_tts(google_api_key, tmap_api_key)



📡 현재 GPS 위치를 가져오는 중...
📍 현재 위치: 위도 37.620453166666664, 경도 127.06065716666667

📍 현재 위치부터 광운대까지 대중교통 경로 안내를 시작합니다.
현재 위치부터 광운대까지 총 소요시간은 9분입니다. 탑승할 노선은 261번입니다. 안내를 시작합니다.
1. 보행자도로 을 따라 54m 이동 (0.3 km, 5분)
2. 좌회전 후 67m 이동 (0.3 km, 5분)
3. 세븐일레븐 광운대후문점 에서 우회전 후 보행자도로 을 따라 56m 이동 (0.3 km, 5분)
4. 우회전 후 165m 이동 (0.3 km, 5분)
5. 나철한의원 에서 좌회전 후 석계로 을 따라 59m 이동 (0.3 km, 5분)
6. 성북치과의원 에서 우측 횡단보도 후 보행자도로 을 따라 22m 이동 (0.3 km, 5분)
7. 코리안바베큐 성북역점 에서 좌회전 후 광운로 을 따라 13m 이동 (0.3 km, 5분)
8. 도착 (0.3 km, 5분)
9. 월계삼거리 정류장에서 261번 버스을(오후 1:42 출발) 탑승하여 1개 정거장을 이동하고, 오후 1:43에 광운대학교 정류장에서 하차합니다. 예상 소요 시간은 2분입니다.
10. 광운로 을 따라 18m 이동 (0.2 km, 3분)
11. 리더스클럽 광운대점 에서 좌측 횡단보도 후 보행자도로 을 따라 20m 이동 (0.2 km, 3분)
12. 광운대학교우체국 에서 우회전 후 광운로 을 따라 23m 이동 (0.2 km, 3분)
13. 광운대학교우체국 에서 좌회전 후 보행자도로 을 따라 158m 이동 (0.2 km, 3분)
14. 도착 (0.2 km, 3분)

🗺️ 경로를 지도에서 확인하려면 아래 링크를 여세요:
https://www.google.com/maps/dir/37.620453166666664%2C127.06065716666667/37.6194277%2C127.05982


# 실시간 경로 안내

In [4]:
import serial
import requests
import re
import time
import math

# ✅ 위도/경도 파싱
def convert_to_decimal(degree_str, direction):
    if not degree_str:
        return None
    if direction in ['N', 'S']:
        d, m = int(degree_str[:2]), float(degree_str[2:])
    elif direction in ['E', 'W']:
        d, m = int(degree_str[:3]), float(degree_str[3:])
    else:
        return None
    result = d + m / 60
    if direction in ['S', 'W']:
        result *= -1
    return result

# ✅ 시리얼 GPS 파싱
def get_gps_coords(ser, timeout_sec=10):
    start_time = time.time()
    while time.time() - start_time < timeout_sec:
        line = ser.readline().decode('utf-8', errors='ignore').strip()
        if line.startswith('$GNRMC') or line.startswith('$GNGGA'):
            try:
                parts = line.split(',')
                if parts[0] == '$GNRMC' and parts[2] == 'A':
                    lat = convert_to_decimal(parts[3], parts[4])
                    lon = convert_to_decimal(parts[5], parts[6])
                    return lat, lon
                elif parts[0] == '$GNGGA' and parts[6] in ['1', '2']:
                    lat = convert_to_decimal(parts[2], parts[3])
                    lon = convert_to_decimal(parts[4], parts[5])
                    return lat, lon
            except Exception:
                continue
    print("❌ GPS 유효 데이터 수신 실패 (타임아웃)")
    return None, None

# ✅ 주소 → 위경도
def address_to_coord_google(address, google_api_key):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": address, "key": google_api_key}
    res = requests.get(url, params=params).json()
    try:
        loc = res["results"][0]["geometry"]["location"]
        return loc["lat"], loc["lng"]
    except:
        print(f"❌ 주소 변환 실패: {address}")
        return None, None

# ✅ 거리 계산 함수 (m 단위)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

# ✅ 경로 좌표 (전체 polyline)
def extract_path_coords(features):
    coords = []
    for f in features:
        if 'geometry' in f and f['geometry']['type'] == 'LineString':
            coords.extend(f['geometry']['coordinates'])
    return coords

# ✅ 경로 이탈 여부 (30m 이상 이탈 시)
def is_off_route(current_lat, current_lon, path_coords, threshold_m=30):
    for lon, lat in path_coords:  # 좌표 순서 주의
        if haversine(current_lat, current_lon, lat, lon) < threshold_m:
            return False
    return True

# ✅ 실시간 도보 길찾기 시스템
def run_live_navigation(google_api_key, tmap_api_key):
    end_name = input("도착지를 입력하세요: ").strip()

    print("\n📡 GPS 초기 위치를 가져오는 중...")
    try:
        ser = serial.Serial("COM3", 9600, timeout=1)  # 포트 필요 시 수정
    except serial.SerialException as e:
        print(f"❌ 시리얼 포트 열기 실패: {e}")
        return

    start_lat, start_lon = get_gps_coords(ser)
    if not start_lat:
        print("❌ GPS 위치 획득 실패")
        return

    end_lat, end_lon = address_to_coord_google(end_name, google_api_key)
    if not end_lat:
        print("❌ 도착지 좌표 변환 실패")
        return

    print(f"🗺️ 경로를 계산 중...")
    route_res = requests.post(
        "https://apis.openapi.sk.com/tmap/routes/pedestrian",
        headers={"appKey": tmap_api_key, "Content-Type": "application/json"},
        json={
            "startX": str(start_lon), "startY": str(start_lat),
            "endX": str(end_lon), "endY": str(end_lat),
            "reqCoordType": "WGS84GEO", "resCoordType": "WGS84GEO",
            "startName": "출발지", "endName": end_name
        }
    ).json()

    if "features" not in route_res:
        print("❌ Tmap 경로 요청 실패")
        return

    # 전체 경로 좌표
    path_coords = extract_path_coords(route_res["features"])

    # 안내할 지점 (Point 타입의 좌표 + description 추출)
    step_points = []
    for f in route_res["features"]:
        desc = f["properties"].get("description")
        geom = f.get("geometry", {})
        if desc and geom.get("type") == "Point":
            lat, lon = geom["coordinates"][1], geom["coordinates"][0]
            step_points.append((desc, lat, lon))

    print("\n🧭 실시간 경로 안내 시작!")
    step_index = 0
    already_said = False

    while step_index < len(step_points):
        cur_lat, cur_lon = get_gps_coords(ser)
        if not cur_lat:
            continue

        # 이탈 감지
        if is_off_route(cur_lat, cur_lon, path_coords):
            print("❗ 경로를 이탈했습니다. 재탐색 중...")
            ser.close()
            run_live_navigation(google_api_key, tmap_api_key)  # 재탐색
            return

        desc, target_lat, target_lon = step_points[step_index]
        distance = haversine(cur_lat, cur_lon, target_lat, target_lon)

        if distance < 20:  # 도달
            print(f"➡️ {step_index + 1}. {desc}")
            step_index += 1
            already_said = False
        else:
            if not already_said:
                # 디버깅용 출력 (TTS 포함 X)
                print(f"🚶 이동 중... {int(distance)}m 남음")
                already_said = True

        time.sleep(2)

    print("🎉 목적지에 도착했습니다!")
    ser.close()


In [ ]:
google_api_key = "abcdef1234567890abcdef1234"  # Google Maps API 키
tmap_api_key = "abcdef1234567890abcdef1234"  # Tmap API 키
   
run_live_navigation(google_api_key, tmap_api_key)



📡 GPS 초기 위치를 가져오는 중...
❌ 시리얼 포트 열기 실패: could not open port 'COM3': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)
